In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,18.05,63,20,2.11,CL,1738509219
1,1,tazovsky,67.4667,78.7000,-29.72,93,13,3.85,RU,1738509220
2,2,touros,-5.1989,-35.4608,30.94,58,59,6.61,BR,1738509221
3,3,kavaratti,10.5669,72.6420,27.62,77,80,3.26,IN,1738509223
4,4,papatowai,-46.5619,169.4708,9.66,76,100,3.24,NZ,1738509224


In [3]:
#Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. 
# The size of the point should be the humidity in each city.
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    title = "Map of Cities",
    frame_width = 800,
    frame_height = 600    
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
# Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp1 = city_data_df[city_data_df["Max Temp"] < 25] # temperature less than 25 degress C
ideal_temp2 = ideal_temp1[ideal_temp1["Max Temp"] < 15 ] # temperature greater than 15 degress C
ideal_temp3 = ideal_temp2[ideal_temp2["Cloudiness"] == 0] # cloudiness = 0
ideal_temp4 = ideal_temp3[ideal_temp3["Wind Speed"] < 2.5] #wind speed less than 2.5
ideal_cities = ideal_temp4[ideal_temp4["Humidity"] < 70] #humidity less than 70%

# Display sample data
ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,bethel,41.3712,-73.4140,-6.55,55,0,2.06,US,1738509000
176,176,pike road,32.2843,-86.1030,10.93,37,0,1.18,US,1738509422
290,290,burang,30.2500,81.1667,-16.34,37,0,1.65,CN,1738509557
301,301,hotan,37.0997,79.9269,-1.54,39,0,1.54,CN,1738509571
334,334,adiyaman,37.7500,38.2500,12.99,26,0,2.06,TR,1738509611


In [5]:
# Step 3: Create a new DataFrame called hotel_df
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
38,bethel,US,41.3712,-73.4140,55,
176,pike road,US,32.2843,-86.1030,37,
290,burang,CN,30.2500,81.1667,37,
301,hotan,CN,37.0997,79.9269,39,
334,adiyaman,TR,37.7500,38.2500,26,


In [6]:
# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.
radius = 10000
limit = 1
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
pike road - nearest hotel: Sleep Inn & Suites East Chase
burang - nearest hotel: No hotel found
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
adiyaman - nearest hotel: Bozdoğan Otel
cabo san lucas - nearest hotel: Comfort Rooms
qamdo - nearest hotel: 银杏商务宾馆
hawkinsville - nearest hotel: No hotel found
at-bashi - nearest hotel: Tash Rabat
torreon - nearest hotel: Holiday Inn Express


,City,Country,Lat,Lng,Humidity,Hotel Name
38,bethel,US,41.3712,-73.4140,55,Hampton Inn Danbury
176,pike road,US,32.2843,-86.1030,37,Sleep Inn & Suites East Chase
290,burang,CN,30.2500,81.1667,37,No hotel found
301,hotan,CN,37.0997,79.9269,39,Hua Yi International Hotel (accepts foreigners)
334,adiyaman,TR,37.7500,38.2500,26,Bozdoğan Otel
408,cabo san lucas,MX,22.8909,-109.9124,38,Comfort Rooms
440,qamdo,CN,31.1667,97.2333,28,银杏商务宾馆
475,hawkinsville,US,32.2838,-83.4721,59,No hotel found
477,at-bashi,KG,41.1702,75.8106,59,Tash Rabat
503,torreon,MX,25.5500,-103.4333,37,Holiday Inn Express


In [7]:
# Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    hover_cols=['Hotel Name', 'Country'],  # Specify columns to show in hover
    color = "City",
    title = "Map of Hotels",
    frame_width = 800,
    frame_height = 600    
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)